# M3 Project

In this project, you will compare the performance of uninformed and informed search algorithms for the 4-sided dominoes problem. You will be given N<sup>2</sup> tiles (N = {2,3,4}) and will be asked to arrange them in an NxN grid in a way that adjacent tiles have the same number in their neighboring sides. The tiles cannot be rotated. See the example below for N=3.

<img src="m3project.png" width="400"/>

An initial version of the code with the problem specification (below) and a report template (at the bottom) are available in this notebook. Deliverables are the final code (non-functioning code is worth 0 points) and the comparison report.

Solve the task above using:
- one uninformed search algorithm of your choice (20pts)
- one informed search algorithm of your choice (20pts)

For your solution, describe the:
- search state space (10pts)
- successor function (10pts)
- heuristic function for the informed search (10pts)

Run each algorithm with at least 10 different initial states for each value of N, and compare the performance of the chosen algorithms for different puzzle sizes in terms of:
- average number of expanded states (15pts)
- success rate (15pts)

You are free to set a maximum number of expanded states for each algorithm, and return failure in case this number is reached.

# Implementation

You are free to change the code below as needed.

In [3]:
import random

In [72]:
class Domino():
    """
    Implementation of a 4-sided domino tile.

    Methods
    -------
    is_above(other)
        Checks of the tile is above the other tile.
    is_under(other)
        Checks of the tile is under the other tile.
    is_on_the_left_of(other)
        Checks of the tile is on the left of the other tile.
    is_on_the_right_of(other)
        Checks of the tile is on the right of the other tile.
    """
    def __init__(self, top: int, right: int, bottom: int, left: int):
        assert isinstance(top, int) and isinstance(right, int) and isinstance(bottom, int) and isinstance(left, int), "Invalid tile value!"
        self.top = top
        self.right = right
        self.bottom = bottom
        self.left = left

    def is_above(self, other):
        assert isinstance(other, Domino), "Invalid tile type!"
        return self.bottom == other.top
    
    def is_under(self, other):
        assert isinstance(other, Domino), "Invalid tile type!"
        return self.top == other.bottom
    
    def is_on_the_left_of(self, other):
        assert isinstance(other, Domino), "Invalid tile type!"
        return self.right == other.left
    
    def is_on_the_right_of(self, other):
        assert isinstance(other, Domino), "Invalid tile type!"
        return self.left == other.right

In [74]:
"""
This implementation is provided as a starting point. Feel free to change it as needed.
"""
class FourDominoes():
    """
    Implementation of the 4-sided dominoes puzzle.

    Methods
    -------
    show()
        Visualize the current state.
    move(action)
        Apply an action to the current state.
    successor(state)
        Finds the list of successors for a given state.
    goal_test(state)
        Checks if the current state is a goal state.
     bfs()
        Performs a Breadth-First Search (BFS) to find the goal state.
    """

  NUM_ITERATIONS = 10
  PUZZLE_SIZES = range(2, 4)

expanded_states = {'BFS': {},'A*': {}}
success_rate = {'BFS': {},'A*': {}}

for N in PUZZLE_SIZES:
    expanded_states['BFS'][N] = 0
    expanded_states['A*'][N] = 0
    success_rate['BFS'][N] = 0
    success_rate['A*'][N] = 0

    for _ in range(NUM_ITERATIONS):
        for algorithm in ['BFS', 'A*']:
            x = FourDominoes(N)
            path = []

            if algorithm == 'BFS':
                path = x.bfs()
            elif algorithm == 'A*':
                path = x.astar()

            if len(path) > 0:
                expanded_states[algorithm][N] += len(path)
                success_rate[algorithm][N] += 1

    expanded_states['BFS'][N] //= NUM_ITERATIONS
    expanded_states['A*'][N] //= NUM_ITERATIONS
    success_rate['BFS'][N] /= NUM_ITERATIONS
    success_rate['A*'][N] /= NUM_ITERATIONS

    print('Performance Comparison')
    print('----------------------')
    for N in PUZZLE_SIZES:
        print('{}x{} grid of tiles'.format(N, N))
        print('BFS - Average Expanded States:', expanded_states['BFS'][N])
        print('A* - Average Expanded States:', expanded_states['A*'][N])
        print('BFS - Success Rate:', success_rate['BFS'][N])
        print('A* - Success Rate:', success_rate['A*'][N])
        print()
    def __init__(self, N, state=None):
        """
        Parameters
        ----------
        N
            Grid size (puzzle contains N^2 tiles)
        state
            Initial state configuration. If None is provided, a random one is created.
        """
        assert N >= 2 and N <= 4, "Invalid grid size!"
        self.N = N

        if state is not None:
            assert len(state) == self.N, "Invalid state size!"
            for row in state:
                assert len(row) == self.N, "Invalid state size!"
                for tile in row:
                    assert isinstance(tile, Domino), "Invalid state type!"
            self.state = state
        else:
            self.state = self.__get_random_state()
    
    def __get_random_state(self):
        """
        Generates a random puzzle configuration (grid of dominoes)

        Return
        ----------
        tuple
            A tuple describing a unique puzzle configuration.
        """
        temp = []
        for i in range(self.N):
            for j in range(self.N):
                domino = Domino(
                    random.randint(1,9) if i == 0 else temp[(i-1)*self.N+j].bottom, # top
                    random.randint(1,9),                                            # right
                    random.randint(1,9),                                            # bottom
                    random.randint(1,9) if j == 0 else temp[i*self.N+j-1].right     #left
                )
                temp.append(domino)
        random.shuffle(temp) # if you comment this line, the state will be a final state (solution)
        return tuple(tuple(temp[i*self.N:(i+1)*self.N]) for i in range(self.N))

    def show(self):
        """
        Prints the current state.
        """
        #line
        print('╔', end='')
        for i in range(self.N):
            print('═══', end='╦' if i < self.N-1 else '╗\n')
        for i in range(self.N):
            # first line of tile
            print('║', end='')
            for j in range(self.N):
                print('╲{}╱║'.format(self.state[i][j].top), end='' if j < self.N-1 else '\n')
            # second line of tile
            print('║', end='')
            for j in range(self.N):
                print('{}╳{}║'.format(self.state[i][j].left, self.state[i][j].right), end='' if j < self.N-1 else '\n')
            # third line of tile
            print('║', end='')
            for j in range(self.N):
                print('╱{}╲║'.format(self.state[i][j].bottom), end='' if j < self.N-1 else '\n')
            # line
            print('╠' if i < self.N-1 else '╚', end='')
            for j in range(self.N):
                print('═══', end='╬' if i < self.N-1 and j < self.N-1 else '╩' if i == self.N-1 and j < self.N-1 else '╣\n' if i < self.N-1 else '╝\n')

    def move(self, action):
        """
        Uses a given action to update the current state of the puzzle. Assumes the action is valid.

        Parameters
        ----------
        action
            Tuple with coordinates of two tiles to be swapped ((row1,col1), (row2,col2))
        """
        assert len(action) == 2 and all(len(coord) == 2 for coord in action) and all(isinstance(x, int) and x >= 0 and x < self.N for coord in action for x in coord), "Invalid action!"
        (r1, c1), (r2, c2) = action
        temp = [list(row) for row in self.state]
        temp[r1][c1], temp[r2][c2] = temp[r2][c2], temp[r1][c1]
        self.state = tuple(tuple(x) for x in temp)

    def successor(self, state):
        """
        Finds the list of successors for a given state.

        Parameters
        ----------
        state
            A tuple describing a unique puzzle configuration.

        Returns
        -------
        list
            A list of pairs (action,state) with all states that can be reached from the given state with a single action.
        """

        successors = []
        for i in range(self.N*self.N):
            r1 = i//self.N
            c1 = i%self.N
            for j in range(i+1, self.N*self.N):
                r2 = j//self.N
                c2 = j%self.N
                action = ((r1,c1), (r2,c2))
                copy = FourDominoes(self.N, state)
                copy.move(action)
                successors.append((action,copy.state))
        return successors

    def goal_test(self, state):
        """
        Checks if the given state is a goal state.

        Parameters
        ----------
        state
            A tuple describing a unique puzzle configuration.

        Returns
        -------
        bool
             True if the given state is a goal state, and False otherwise.
        """

        for i in range(self.N):
            for j in range(self.N):
                if i > 0 and not state[i][j].is_under(state[i-1][j]):
                    return False
                if j > 0 and not state[i][j].is_on_the_right_of(state[i][j-1]):
                    return False
        return True
    
        def bfs(self):
    """
    Performs Breadth-First Search on the puzzle starting from the initial state.

    Returns
    -------
    list or None
        If a goal state is found, returns a list of actions to reach the goal state.
        If no goal state is found, returns None.
    """
    queue = [(self.state, [])]  # Initialize the queue with the initial state and an empty action list

    while queue:
        state, actions = queue.pop(0)  # Get the state and actions from the front of the queue

        if self.goal_test(state):  # Check if the current state is a goal state
            return actions

        successors = self.successor(state)  # Get the list of successors for the current state

        for action, successor_state in successors:
            queue.append((successor_state, actions + [action]))  # Add each successor state and its corresponding action sequence to the queue

    return None  # If no goal state is found, return None
    for N in range(2, 5):
    print('-----------------')
    print('{}x{} grid of tiles'.format(N, N))
    print('-----------------')

    x = FourDominoes(N)
    x.show()

    print('This state has {} successors!'.format(len(x.successor(x.state))))
    if x.goal_test(x.state):
        print('This state is a goal state!')
    else:
        print('This state is not a goal state!')

    x.move(((0, 0), (1, 1)))
    x.show()

    print('This state has {} successors!'.format(len(x.successor(x.state))))
    if x.goal_test(x.state):
        print('This state is a goal state!')
    else:
        print('This state is not a goal state!')

    print('Performing BFS...')
    solution = x.bfs()
    if solution:
        print('Found a solution!')
        print('Actions:', solution)
    else:
        print('No solution found.')

    print() 
    
    def heuristic(self, state):
        # Heuristic function implementation...
        pass

    def astar(self):
        # A* search implementation...
        frontier = PriorityQueue()
        frontier.put((0, self.state))  # Start state with priority 0
        came_from = {}
        cost_so_far = {}
        came_from[self.state] = None
        cost_so_far[self.state] = 0
    def show_state(self, state):
        # Helper method to visualize a specific state...
        pass
 # Call astar to obtain the path from the start state to the goal state
    path = x.astar()
    print('Path from start to goal:')
    for action, state in path:
        print('Action:', action)
        x.show_state(state)
        print()

    print()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 22)

In [61]:
for N in range(2,5):
    print('-----------------')
    print('{}x{} grid of tiles'.format(N,N))
    print('-----------------')

    x = FourDominoes(N)
    x.show()

    print('This state has {} successors!'.format(len(x.successor(x.state))))
    if(x.goal_test(x.state)):
        print('This state is a goal state!')
    else:
        print('This state is not a goal state!')

    x.move(((0,0),(1,1)))
    x.show()

    print('This state has {} successors!'.format(len(x.successor(x.state))))
    if(x.goal_test(x.state)):
        print('This state is a goal state!')
    else:
        print('This state is not a goal state!')
    
    print()

-----------------
2x2 grid of tiles
-----------------
╔═══╦═══╗
║╲3╱║╲6╱║
║7╳5║5╳5║
║╱9╲║╱7╲║
╠

║╲4╱║╲6╱║
║2╳2║2╳2║
║╱3╲║╱6╲║
╚

This state has 0 successors!
This state is a goal state!


AttributeError: 'FourDominoes' object has no attribute 'move'

# Report template

## Solution description

Methods implemented, etc. Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Scelerisque eu ultrices vitae auctor eu augue. Tempor nec feugiat nisl pretium fusce. Sed cras ornare arcu dui vivamus arcu felis. Leo duis ut diam quam nulla porttitor massa. Lacus vestibulum sed arcu non odio euismod lacinia. Augue neque gravida in fermentum et sollicitudin ac orci. Habitant morbi tristique senectus et. Ut tellus elementum sagittis vitae et. Diam vulputate ut pharetra sit amet. Nisl nunc mi ipsum faucibus vitae aliquet nec ullamcorper sit. Nec ullamcorper sit amet risus nullam eget felis. Fringilla ut morbi tincidunt augue interdum velit euismod. Facilisis magna etiam tempor orci eu lobortis.

### Search space

Format, size, start, goal, etc. Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Scelerisque eu ultrices vitae auctor eu augue. Tempor nec feugiat nisl pretium fusce. Sed cras ornare arcu dui vivamus arcu felis. Leo duis ut diam quam nulla porttitor massa. Lacus vestibulum sed arcu non odio euismod lacinia. Augue neque gravida in fermentum et sollicitudin ac orci. Habitant morbi tristique senectus et. Ut tellus elementum sagittis vitae et. Diam vulputate ut pharetra sit amet. Nisl nunc mi ipsum faucibus vitae aliquet nec ullamcorper sit. Nec ullamcorper sit amet risus nullam eget felis. Fringilla ut morbi tincidunt augue interdum velit euismod. Facilisis magna etiam tempor orci eu lobortis.

### Successor function

How it works? Average number of successors? Etc. Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Scelerisque eu ultrices vitae auctor eu augue. Tempor nec feugiat nisl pretium fusce. Sed cras ornare arcu dui vivamus arcu felis. Leo duis ut diam quam nulla porttitor massa. Lacus vestibulum sed arcu non odio euismod lacinia. Augue neque gravida in fermentum et sollicitudin ac orci. Habitant morbi tristique senectus et. Ut tellus elementum sagittis vitae et. Diam vulputate ut pharetra sit amet. Nisl nunc mi ipsum faucibus vitae aliquet nec ullamcorper sit. Nec ullamcorper sit amet risus nullam eget felis. Fringilla ut morbi tincidunt augue interdum velit euismod. Facilisis magna etiam tempor orci eu lobortis.

### Heuristic function

How it works? Admissible? Etc. Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Scelerisque eu ultrices vitae auctor eu augue. Tempor nec feugiat nisl pretium fusce. Sed cras ornare arcu dui vivamus arcu felis. Leo duis ut diam quam nulla porttitor massa. Lacus vestibulum sed arcu non odio euismod lacinia. Augue neque gravida in fermentum et sollicitudin ac orci. Habitant morbi tristique senectus et. Ut tellus elementum sagittis vitae et. Diam vulputate ut pharetra sit amet. Nisl nunc mi ipsum faucibus vitae aliquet nec ullamcorper sit. Nec ullamcorper sit amet risus nullam eget felis. Fringilla ut morbi tincidunt augue interdum velit euismod. Facilisis magna etiam tempor orci eu lobortis.

## Experimental results

Results are presented in the table below. Each search was repeated 10 times for each implemented method. The maximum number of expanded states per run was set to 1000000. **Avg** shows the average number of expanded states, **Goal** shows the number of times the goal was reached in each case, **Fail #1** shows the number of times the search ended without reaching the goal, and **Fail #2** shows the number of times the search ended because the maximum number of expanded states was reached. Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Scelerisque eu ultrices vitae auctor eu augue. Tempor nec feugiat nisl pretium fusce. Sed cras ornare arcu dui vivamus arcu felis. Leo duis ut diam quam nulla porttitor massa. Lacus vestibulum sed arcu non odio euismod lacinia. Augue neque gravida in fermentum et sollicitudin ac orci. Habitant morbi tristique senectus et. Ut tellus elementum sagittis vitae et. Diam vulputate ut pharetra sit amet. Nisl nunc mi ipsum faucibus vitae aliquet nec ullamcorper sit. Nec ullamcorper sit amet risus nullam eget felis. Fringilla ut morbi tincidunt augue interdum velit euismod. Facilisis magna etiam tempor orci eu lobortis.

| Puzzle size |&#124;| Uninformed | |         |         |&#124;| Informed | |         |         |
|-------------|------|-------|------|---------|---------|------|-----|------|---------|---------|
|             |&#124;| Avg   | Goal | Fail #1 | Fail #2 |&#124;| Avg | Goal | Fail #1 | Fail #2 |
| 2x2         |&#124;| 1     | 10   | 0       | 0       |&#124;| 1   | 10   | 0       | 0       |
| 3x3         |&#124;| 100   | 3    | 7       | 0       |&#124;| 10  | 7    | 3       | 0       |
| 4x4         |&#124;| 10000 | 0    | 2       | 8       |&#124;| 100 | 3    | 4       | 3       |